## In this notebook i will create and train the first prediction model

In [ ]:
#this code reads the dataframe from the saved csv
import pandas as pd
df = pd.read_csv(
    r"C:\Users\ADMIN\Desktop\Coding_projects\stock_market_prediction\Stock-Market-Prediction\data\processed\modified_btc_data.csv",
    index_col=0,
    parse_dates=True
)
#print(df)


In [7]:
#This code adds a column for the next hour price ,
#  in order for us to do the prediction calculation
df["next_hour"] = df["close"].shift(-1)

In [ ]:
df

In [9]:
# This code checks if the price went up from a ceratin hour to its next hour
# and the  result type was converted to int for our ML model.
df["Target"] = (df["next_hour"] > df["close"]).astype(int)

In [ ]:
#This code removes all the data that came before a certain date 
#(I used a copy of our dataframe just to check that it works) , the code for our df is 
# df=df.loc["2016-01-30":].copy()
dfcopy=df.loc["2016-01-30":].copy()
#print(dfcopy)

In [18]:
print(dfcopy)

                             open      high       low     close    Volume BTC  \
date                                                                            
2016-01-30 00:00:00    378.942811    379.00    376.55    379.00  3.276797e+01   
2016-01-30 01:00:00    376.800000    378.50    376.34    376.34  9.129230e+00   
2016-01-30 02:00:00    378.500000    378.50    378.50    378.50  1.503970e-01   
2016-01-30 03:00:00    376.510000    376.51    376.51    376.51  6.277551e-02   
2016-01-30 04:00:00    376.510000    380.50    374.00    380.50  2.718272e+02   
...                           ...       ...       ...       ...           ...   
2025-03-23 20:00:00  84963.370000  85224.99  84963.37  85066.48  3.662315e+06   
2025-03-23 21:00:00  85052.360000  85166.93  84900.02  85163.56  3.638842e+06   
2025-03-23 22:00:00  85163.840000  85635.21  85142.67  85325.09  3.713001e+06   
2025-03-23 23:00:00  85326.050000  86071.85  85325.38  86048.22  3.631318e+06   
2025-03-24 00:00:00  86048.2

In [ ]:

#This code sets the learning paramters and trains the model

# n_estimators -? the number of decision tress we want to learn (higher = better accuarcy up to a limit)
# 100 is a low number - need to exepiremnt with it to find the correct number

# Min_samples_split -> helps us protect aginst overfitting (if the build the tree too deeply) 
# - need to exepiremnt with it to find the correct number

#random state -> if we run the same model twice the random numbers that are genreated will be in a predictable
# sequance each time , using this random seed of 1 (if we run the same model twice we will get same results)

from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=100,min_samples_split=100,random_state=1)

train = dfcopy.iloc[:-100] #All the rows except the last 100 rows will be put in the training set
test = dfcopy.iloc[:-100] # The last 100 rows will be put in the test set

# list of all the columns we are going to use to predict the target
predictors = ["close","Volume BTC","open","high","low"]

#This is going to train the model to predict the target using the predictor columns
model.fit(train[predictors],train["Target"])



RandomForestClassifier(min_samples_split=100, random_state=1)

In [12]:
#This code will check our model , (when we said the market would go up did it acctualy go up)
from sklearn.metrics import precision_score

test = dfcopy.iloc[:-100] # The last 100 rows will be put in the test set
predictors = ["close","Volume BTC","open","high","low"]
#this will generate predictions using our model
preds = model.predict(test[predictors])


In [13]:
# this will convert our predictions into a series fromat (easier to read)
import pandas as pd 
preds = pd.Series(preds,index=test.index) 

In [ ]:
#show the prediction array
#preds

In [14]:
#Calculation of the precission score
precision_score(test["Target"],preds)

0.7617517256987666

In [26]:
#this code will save the model so u can keep working on it later
import os
import joblib

# Ensure the directory exists
os.makedirs('models', exist_ok=True)

# Save the model
joblib.dump(model, 'models/random_forest_model.pkl')



['models/random_forest_model.pkl']

In [1]:
#Load the model back up 
import joblib

# Load the saved model from file
model = joblib.load('models/random_forest_model.pkl')


In [19]:
# this function wraps up everything we did before into a function
import pandas as pd
def predict(train, test, predictors, model):
    model.fit(train[predictors], train["Target"]) #fitting of the model using the trainig and target
    preds = model.predict(test[predictors]) #genrating predictions
    preds = pd.Series(preds, index=test.index, name="Predictions") #combining hte model into a series
    combined = pd.concat([test["Target"], preds], axis=1) # combinig everything togehter to a data frame
    return combined



In [20]:
# a backtesting function , which takes the data , ml model , preddictors , start and step value
#start = the amount of that data that will train the model
# start=48000 - > learn on the first 48000 rows (first 8 years of data)
#step = the amount of data that will be learned in each step (we will go from a year to another year)
def backtest(data, model, predictors, start=48000, step=6000):
    all_predictions = [] # list of dataframes of predictions for a singel year

    for i in range(start, data.shape[0], step): # loop across the data year by year
        train = data.iloc[:i].copy() #(writing .copy to avoid the copy warning)
        test = data.iloc[i:i+step].copy() 
        predictions = predict(train, test, predictors, model) # predict function to genreate predictions
        all_predictions.append(predictions) #append the predictions for a giving year

    return pd.concat(all_predictions) #concat all the dataframes togheter


In [21]:
#backtest the data with the model and predictors we calcaulted earlier
predictions = backtest(dfcopy, model, predictors)


In [24]:
#check how many days we predicted the market would go up vs down 
#value_count will count how many times each type of prediction was made
predictions["Predictions"].value_counts()


Predictions
0    18970
1    13213
Name: count, dtype: int64

In [ ]:
# look at the precision score
precision_score(predictions["Target"],predictions["Predictions"])

#0.5085900249754031



0.5085900249754031

In [ ]:
#benchmark to check the prectange of days where the market acctualy went up
predictions["Target"].value_counts() / predictions.shape[0]
 
#1    0.505484
#0    0.494516

In [ ]:
#create a variety of rolling avreges , to help identify 
#trends of wheter the stock will go up or down

# trends for the last 2 days , 5 days , 60 days , 250 days , 1000 days
horizons = [24*2,24*5,24*60,24*250,24*1000]
new_predictors = []

for horizon in horizons: # we will loop through the horizions and calcaulte a rolling avrege
    rolling_averages = dfcopy.rolling(horizon).mean()

    #Column for that will contatin the close value of each horizion
    ratio_column = f"close_ratio_{horizon}"

    #calculating the ratio bettwen our current close value and each of the horzions
    #(and adding it to the data frame)
    dfcopy[ratio_column] = dfcopy["close"] / rolling_averages["close"]

    #the number of hours in the current horrizion that the stock price went up
    trend_column = f"Trend_{horizon}"
    dfcopy[trend_column] = dfcopy.shift(1).rolling(horizon).sum()["Target"]

    new_predictors += [ratio_column,trend_column]


In [ ]:
dfcopy
# why we have Nans?
#when panads cant find enough rows prior to our current row in order to compute the rolling avrege it gives us nan


In [ ]:
#drop the rows where we have na in the columns
dfcopy = dfcopy.dropna()

#we can see now that we dropped a few years since we needed them to calcualte hte trend 1000 and close 1000 days of trading


In [ ]:
model = RandomForestClassifier(n_estimators=200,min_samples_split=50,random_state=1)

In [ ]:
# changing a little bit our predict fucntion that now will return the probability that the stoock price would go up or down

import pandas as pd
def predict(train, test, predictors, model):
    model.fit(train[predictors], train["Target"]) 
    preds = model.predict_proba(test[predictors])[:,1] #we get the second column of this (the prob that the stock would go up) 
    #now we would set a trehshold that only if the probability is 60% to go up then the model would predict the increase
    preds[preds>=.6] = 1 # it will reduce the number of days we can predict the price to go up , but we would increase the the prediction accuracy
    preds[preds<.6] = 0
    preds = pd.Series(preds, index=test.index, name="Predictions") 
    combined = pd.concat([test["Target"], preds], axis=1) 
    return combined
